In [18]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("stabilityai/japanese-stablelm-instruct-gamma-7b")
model = AutoModelForCausalLM.from_pretrained(
  "stabilityai/japanese-stablelm-instruct-gamma-7b",
  torch_dtype=torch.half
)
model.eval()

if torch.cuda.is_available():
    model = model.to("cuda")

def build_prompt(user_query, inputs="", sep="\n\n### "):
    # sys_msg = "あなたは国語教師です。以下の問題を考え、正しい選択肢を説明してください。"
    sys_msg = "以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。"
    p = sys_msg
    roles = ["指示", "応答"]
    msgs = [": \n" + user_query, ": \n"]
    if inputs:
        roles.insert(1, "入力")
        msgs.insert(1, ": \n" + inputs)
    for role, msg in zip(roles, msgs):
        p += sep + role + msg
    return p


Loading checkpoint shards: 100%|██████████| 3/3 [00:07<00:00,  2.44s/it]


In [6]:
import pandas as pd
N3 = pd.read_csv('./N3_answers.csv')
N3

,Question,Options,Answer,Answer_Option
0,今日は___寒くないので、上着はいらないと思う。,a.ただ b.たいして c.なんとかして,a.ただ,['a']
1,今日は___品物を見ていただけで、何も買わなかった。,a.ただ b.いくら c.せっかく,a.,['a']
2,___怒らないでわたしの話を聞いてください。,a.どうか b.どうも c.どんなに,d.どうぞ,[]
3,転んだ子供は今にも___顔をしていた。,a.泣いた b.泣きそうな c.泣くような,b.,['b']
4,せっかくカメラを___、みんなで写真を撮ろう。,a.持ってきて b.持ってきたら c. 持ってきたのだから,a.持ってきて,['a']
...,...,...,...,...
79,先生の説明を聞いている___だんだんわかってきた。,a.うちに b.たびに c.ところを,c.,['c']
80,今朝___喉が痛かった。,a.起きたとき b.起きたついでに c.起きてるうちに,b.起きたついでに\n\n### 指示: \n文法を基づいて、最もよい,['b']
81,祖父は年を取って、最近___なかった。今日もかぎの場所を忘れた。,a.忘れかけに b.忘れにくく c.忘れっぽく,A.b 「忘れやすい」を形容詞化した表現だが、「忘れっぽ,['b']
82,この店には高い品物___並んでいる。,a.ぐらい b.なんか c.しか d.ばかり,この店には高い品物ばかり並んでいる。a.ぐらい b.なんか c.,"['a', 'b', 'c']"


In [19]:
def ask_sensei(question, options):
    question = question.replace("___", "[MASK]")
    # Infer with prompt without any additional input
    user_inputs = {
        "user_query":'まず、文法を基づいて、最もよい選択肢のアルファベットを一つ選びなさい。次に、文法について、十五字以内説明しなさい。',
        "inputs": f"問題：{question}　選択肢：{options}"
    }

    prompt = build_prompt(**user_inputs)

    # print(prompt)
    input_ids = tokenizer.encode(
        prompt, 
        add_special_tokens=True, 
        return_tensors="pt"
    )

    attention_mask = torch.ones_like(input_ids).to('cuda')  # Create attention mask with all 1s for non-padding tokens

    tokens = model.generate(
        input_ids.to(device=model.device),
        attention_mask=attention_mask, 
        max_new_tokens=64,
        temperature=1,
        repetition_penalty=1.1,
        top_p=0.95,
        do_sample=True,
    )
    out = tokenizer.decode(tokens[0][input_ids.shape[1]:], skip_special_tokens=True).strip()
    print(out)
    print("------------")
    return out

In [20]:
sensei_ans = []
for index, row in N3.iterrows():
    answer = ask_sensei(row.Question, row.Options)
    sensei_ans.append(answer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1.選択肢「c.なんとか」が正解であり、これは、主語を「today」（今日）から「Today」（今）に直すことで文意を変えています。同様
------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


選択肢c.せっかく　が一番適しています。　理由は、助詞「せっ」は助動詞（終止形の変化で接続語と意味的な役割を
------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

選択肢aは、誤った解答であり、正当化はできません。この場合、選択肢aがある動詞｢似る｣に対する形容�����
------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1.	選択肢を再構成（前の記事でやった手順）：「子供が食べられる」料理にするには、どのような調理法を試すことができますか？a
------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


男女の差異がある文章です。「b.ように」
------------


In [ ]:
N3['Answer_Update'] = sensei_ans

In [ ]:
import numpy as np
N3['Answer_Option_Update'] = N3['Answer_Update'].str.findall(r'a|b|c')

In [ ]:
pd.set_option('display.max_rows', None)
display(N3)
pd.set_option('display.max_rows', 10)

,Question,Options,Answer,Answer_Option
0,今日は___寒くないので、上着はいらないと思う。,a.ただ b.たいして c.なんとかして,a.ただ,[a]
1,今日は___品物を見ていただけで、何も買わなかった。,a.ただ b.いくら c.せっかく,a.,[a]
2,___怒らないでわたしの話を聞いてください。,a.どうか b.どうも c.どんなに,d.どうぞ,[]
3,転んだ子供は今にも___顔をしていた。,a.泣いた b.泣きそうな c.泣くような,b.,[b]
4,せっかくカメラを___、みんなで写真を撮ろう。,a.持ってきて b.持ってきたら c. 持ってきたのだから,a.持ってきて,[a]
5,3時に駅で待ち合わせした。駅に着くと、彼女はもう___。,a.来た b.来ていた,a,[a]
6,チャイムが鳴ったので玄関の外に出ると、宅配便の人が___。,a.来た b.いた,a.来た,[a]
7,目が痛いので、鏡で___、赤かった。,a.見てみると b.見ていると c.見ていたら,b. 見ていると,[b]
8,今は___お金もない。旅行は無理だ。,a.忙しければ b.暇もなければ c.時間もあれば,c,[c]
9,図書館で古い本は捨てられることに___。,a.した b.なった,a. した,[a]


In [ ]:
a = 'a'
b = 'b'
c = 'c'
d = 'd'
e = 'e'
f = 'f'
g = 'g'

N3_correct_answers = [b,a,a,b,c,b,b,a,b,b,b,b,a,a,a,b,b,b,a,a,d,c,a,b,d,a,a,a,b,b,a,b,a,c,a,b,a,b,b,b,b,b,b,b,c,e,c,b,g,c,a,d,b,b,c,a,c,a,a,b,a,c,c,b,d,a,a,c,a,c,b,a,c,a,c,a,c,b,a,a,c,d,b]
N2_correct_answers = []

In [ ]:
# N3['correct_answer'] = N3_correct_answers
N3.to_csv("./N3_answers.csv", index=False)